<a href="https://colab.research.google.com/github/peterokwara/covid-19-map/blob/master/covid19Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install geopandas library


In [1]:
!pip install geopandas

     |████████████████████████████████| 931kB 2.8MB/s 
     |████████████████████████████████| 14.7MB 272kB/s 
     |████████████████████████████████| 10.9MB 46.6MB/s 


Import the necessary libraries

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Import the covid-19 dataset from github

In [0]:
# Data urls
base_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/'
confirmed_cases_data_url = base_url + 'time_series_covid19_confirmed_global.csv'
death_cases_data_url = base_url + 'time_series_covid19_deaths_global.csv'
recovery_cases_data_url = base_url+ 'time_series_covid19_recovered_global.csv'

# Import datasets as pandas dataframes
raw_data_confirmed = pd.read_csv(confirmed_cases_data_url)
raw_data_deaths = pd.read_csv(death_cases_data_url)
raw_data_recovered = pd.read_csv(recovery_cases_data_url)

Let's check out the confirmed cases

In [5]:
raw_data_confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,94,110,110,120,170,174,237,273,281,299,349,367,423,444,484,521,555,607,665,714,784,840,906,933,996,1026,1092,1176,1279,1351,1463,1531,1703,1828,1939,2171,2335,2469,2704,2894
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,174,186,197,212,223,243,259,277,304,333,361,377,383,400,409,416,433,446,467,475,494,518,539,548,562,584,609,634,663,678,712,726,736,750,766,773,782,789,795,803
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,367,409,454,511,584,716,847,986,1171,1251,1320,1423,1468,1572,1666,1761,1825,1914,1983,2070,2160,2268,2418,2534,2629,2718,2811,2910,3007,3127,3256,3382,3517,3649,3848,4006,4154,4295,4474,4648
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,224,267,308,334,370,376,390,428,439,466,501,525,545,564,583,601,601,638,646,659,673,673,696,704,713,717,717,723,723,731,738,738,743,743,743,745,745,747,748,750
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,4,5,7,7,7,8,8,8,10,14,16,17,19,19,19,19,19,19,19,19,19,19,24,24,24,24,25,25,25,25,26,27,27,27,27,30,35,35,35


Let's group the data by countries, dropping the longitude and latitude data

In [6]:
# Group by region (also drop 'Lat', 'Long' as it doesn't make sense to sum them here)
confirmed_country = raw_data_confirmed.groupby(['Country/Region']).sum().drop(['Lat', 'Long'], axis=1)
confirmed_country.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,...,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20
Country/Region,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,...,94,110,110,120,170,174,237,273,281,299,349,367,423,444,484,521,555,607,665,714,784,840,906,933,996,1026,1092,1176,1279,1351,1463,1531,1703,1828,1939,2171,2335,2469,2704,2894
Albania,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,174,186,197,212,223,243,259,277,304,333,361,377,383,400,409,416,433,446,467,475,494,518,539,548,562,584,609,634,663,678,712,726,736,750,766,773,782,789,795,803
Algeria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,...,367,409,454,511,584,716,847,986,1171,1251,1320,1423,1468,1572,1666,1761,1825,1914,1983,2070,2160,2268,2418,2534,2629,2718,2811,2910,3007,3127,3256,3382,3517,3649,3848,4006,4154,4295,4474,4648
Andorra,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,224,267,308,334,370,376,390,428,439,466,501,525,545,564,583,601,601,638,646,659,673,673,696,704,713,717,717,723,723,731,738,738,743,743,743,745,745,747,748,750
Angola,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,4,5,7,7,7,8,8,8,10,14,16,17,19,19,19,19,19,19,19,19,19,19,24,24,24,24,25,25,25,25,26,27,27,27,27,30,35,35,35


In [8]:
confirmed_country.index

Index(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria',
       ...
       'United Kingdom', 'Uruguay', 'Uzbekistan', 'Venezuela', 'Vietnam',
       'West Bank and Gaza', 'Western Sahara', 'Yemen', 'Zambia', 'Zimbabwe'],
      dtype='object', name='Country/Region', length=187)

Select the last date, as the last date contains the latest, total amount of cases for each country

In [36]:
total_confirmed_cases  = confirmed_country.iloc[:,-1].reset_index()
total_confirmed_cases.columns = [*total_confirmed_cases.columns[:-1], 'Total Cases']
print(total_confirmed_cases.head())
print(total_confirmed_cases.columns)

  Country/Region  Total Cases
0    Afghanistan         2894
1        Albania          803
2        Algeria         4648
3        Andorra          750
4         Angola           35
Index(['Country/Region', 'Total Cases'], dtype='object')


Import the map of Africa and print out the first 5 rows

In [18]:
africa_map = gpd.read_file("https://raw.githubusercontent.com/peterokwara/covid-19-map/master/custom.geo.json")
africa_map.head()

,scalerank,featurecla,labelrank,sovereignt,sov_a3,adm0_dif,level,type,admin,adm0_a3,geou_dif,geounit,gu_a3,su_dif,subunit,su_a3,brk_diff,name,name_long,brk_a3,brk_name,brk_group,abbrev,postal,formal_en,formal_fr,note_adm0,note_brk,name_sort,name_alt,mapcolor7,mapcolor8,mapcolor9,mapcolor13,pop_est,gdp_md_est,pop_year,lastcensus,gdp_year,economy,income_grp,wikipedia,fips_10,iso_a2,iso_a3,iso_n3,un_a3,wb_a2,wb_a3,woe_id,adm0_a3_is,adm0_a3_us,adm0_a3_un,adm0_a3_wb,continent,region_un,subregion,region_wb,name_len,long_len,abbrev_len,tiny,homepart,filename,geometry
0,1,Admin-0 country,3,Burkina Faso,BFA,0,2,Sovereign country,Burkina Faso,BFA,0,Burkina Faso,BFA,0,Burkina Faso,BFA,0,Burkina Faso,Burkina Faso,BFA,Burkina Faso,None,B.F.,BF,Burkina Faso,None,None,None,Burkina Faso,None,2,1,5,11,15746232,17820.0,-99,2006,-99,7. Least developed region,5. Low income,-99,None,BF,BFA,854,854,BF,BFA,-99,BFA,BFA,-99,-99,Africa,Africa,Western Africa,Sub-Saharan Africa,12,12,4,-99,1,BFA.geojson,"POLYGON ((-2.82750 9.64246, -3.51190 9.90033, ..."
1,1,Admin-0 country,4,Republic of Congo,COG,0,2,Sovereign country,Republic of Congo,COG,0,Republic of Congo,COG,0,Republic of Congo,COG,0,Congo,Republic of Congo,COG,Republic of Congo,None,Rep. Congo,CG,Republic of Congo,None,None,None,"Congo, Rep.",None,2,1,3,10,4012809,15350.0,-99,2007,-99,6. Developing region,4. Lower middle income,-99,None,CG,COG,178,178,CG,COG,-99,COG,COG,-99,-99,Africa,Africa,Middle Africa,Sub-Saharan Africa,5,17,10,-99,1,COG.geojson,"POLYGON ((12.99552 -4.78110, 12.62076 -4.43802..."
2,1,Admin-0 country,5,Benin,BEN,0,2,Sovereign country,Benin,BEN,0,Benin,BEN,0,Benin,BEN,0,Benin,Benin,BEN,Benin,None,Benin,BJ,Republic of Benin,None,None,None,Benin,None,1,2,2,12,8791832,12830.0,-99,2002,-99,7. Least developed region,5. Low income,-99,None,BJ,BEN,204,204,BJ,BEN,-99,BEN,BEN,-99,-99,Africa,Africa,Western Africa,Sub-Saharan Africa,5,5,5,-99,1,BEN.geojson,"POLYGON ((2.69170 6.25882, 1.86524 6.14216, 1...."
3,1,Admin-0 country,3,Angola,AGO,0,2,Sovereign country,Angola,AGO,0,Angola,AGO,0,Angola,AGO,0,Angola,Angola,AGO,Angola,None,Ang.,AO,People's Republic of Angola,None,None,None,Angola,None,3,2,6,1,12799293,110300.0,-99,1970,-99,7. Least developed region,3. Upper middle income,-99,None,AO,AGO,024,024,AO,AGO,-99,AGO,AGO,-99,-99,Africa,Africa,Middle Africa,Sub-Saharan Africa,6,6,4,-99,1,AGO.geojson,"MULTIPOLYGON (((16.32653 -5.87747, 16.57318 -6..."
4,1,Admin-0 country,3,Ivory Coast,CIV,0,2,Sovereign country,Ivory Coast,CIV,0,Ivory Coast,CIV,0,Ivory Coast,CIV,0,Côte d'Ivoire,Côte d'Ivoire,CIV,Côte d'Ivoire,None,I.C.,CI,Republic of Ivory Coast,Republic of Cote D'Ivoire,None,None,Côte d'Ivoire,None,4,6,3,3,20617068,33850.0,-99,1998,-99,6. Developing region,4. Lower middle income,-99,None,CI,CIV,384,384,CI,CIV,-99,CIV,CIV,-99,-99,Africa,Africa,Western Africa,Sub-Saharan Africa,13,13,4,-99,1,CIV.geojson,"POLYGON ((-2.85613 4.99448, -3.31108 4.98430, ..."


Plot a chorepleth map of Africa with total number of cases

In [37]:
# choropleth maps bind Pandas Data Frames and json geometries.
# This allows us to quickly visualize data combinations

map = folium.Map(location=[5, 20], zoom_start=3.5)

map.choropleth(geo_data=africa_map, data=total_confirmed_cases, 
               columns=['Country/Region', 'Total Cases'], 
               key_on='feature.properties.name', 
               fill_color='RdPu', fill_opacity=0.75, line_opacity=0.5, 
               legend_name="Total Covid19 Cases")
display(map)

/usr/local/lib/python3.6/dist-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning
